In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk

data = pd.read_csv("word2vec.csv")
data = data.fillna("")
data.head()

,PRODUCT_TYPE_ID,PRODUCT_LENGTH,TEXT,vectors
0,1650,2125.980000,artzfolio tulip flowers blackout curtain door ...,[[ 0.24486599 0.09132601 0.12729792 0.01665...
1,2755,393.700000,marks spencer girls pyjama sets t862561cnavy m...,[[ 0.12145305 0.06881733 0.02220269 0.00592...
2,7537,748.031495,priknik horn red electric air horn compressor ...,[[ 0.03937757 0.08054779 0.1644971 -0.12261...
3,2996,787.401574,alishah womens cotton ankle length leggings co...,[[ 0.21314571 0.09550028 0.01377743 -0.02590...
4,6112,598.424000,united empire loyalists chronicle great migrat...,[[ 0.07313064 0.02169267 -0.10552848 0.11775...


In [2]:
data['vectors'][0]

'[[ 0.24486599  0.09132601  0.12729792  0.01665498 -0.04266906 -0.02752393\n   0.04395749  0.0552298   0.01799644 -0.131849   -0.08165585 -0.04776286\n  -0.158705   -0.09594734  0.03506118  0.05094731  0.02694449  0.00624992\n  -0.0498466  -0.06658363  0.02969561  0.13801886 -0.18002258 -0.09880821\n  -0.11437636  0.05922138 -0.04789598  0.16742549  0.15893989  0.29970212\n  -0.12233958 -0.11272874 -0.12335229 -0.09279434  0.01145658  0.08247568\n   0.09797681  0.01688097 -0.00224198 -0.11139964 -0.04053711 -0.10545407\n   0.06608243  0.05192966  0.03598407  0.04658067 -0.01379358 -0.0969209\n  -0.14873363  0.06211954  0.10641119 -0.00880902  0.12594728  0.00325343\n  -0.00383921  0.09831329 -0.08108894  0.08207243  0.03107267  0.09872813\n  -0.02845263 -0.01513546 -0.13255188  0.13859841 -0.07904432  0.06826977\n   0.10151197 -0.00693005  0.10439347  0.10852824 -0.01616291 -0.07489225\n   0.11050934 -0.0813156   0.10137352  0.06234654  0.18863145 -0.06374631\n   0.05986893  0.10759768

In [4]:
data['TEXT'] = data['TITLE'] + ' ' + data['DESCRIPTION'] + ' ' + data['BULLET_POINTS']
data.drop('TITLE', axis=1, inplace=True)
data.drop('DESCRIPTION', axis=1, inplace=True)
data.drop('BULLET_POINTS', axis=1, inplace=True)

In [10]:
data.to_csv('preprocessed2.csv', index=False)

In [15]:
print(len(data['PRODUCT_TYPE_ID'].unique()))
data.head()

12907


,PRODUCT_TYPE_ID,PRODUCT_LENGTH,TEXT
0,1650,2125.980000,artzfolio tulip flowers blackout curtain door ...
1,2755,393.700000,marks spencer girls pyjama sets t862561cnavy m...
2,7537,748.031495,priknik horn red electric air horn compressor ...
3,2996,787.401574,alishah womens cotton ankle length leggings co...
4,6112,598.424000,united empire loyalists chronicle great migrat...


In [26]:
for col in ['DESCRIPTION', 'TITLE', 'BULLET_POINTS']:
    data[col] = data[col].apply(lambda x: x.lower())

for col in ['DESCRIPTION', 'TITLE', 'BULLET_POINTS']:
    data[col] = data[col].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)))
    
stop = stopwords.words('english')
for col in ['DESCRIPTION', 'TITLE', 'BULLET_POINTS']:
    data[col] = data[col].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

data['TEXT'] = data['TEXT'].apply(lambda x: tokenize(x))

KeyboardInterrupt: 

In [2]:
import nltk
nltk.download('punkt')
from tqdm import tqdm
import pandas as pd
import numpy as np
batch_size = 1000
total = len(data)
pbar = tqdm(total=total, desc='Tokenizing data')
data_batches = np.array_split(data, len(data) // batch_size)
tokenized_data = []
for batch in data_batches:
    batch['TEXT'] = batch['TEXT'].apply(lambda x: nltk.word_tokenize(x))
    tokenized_data.append(batch)
    pbar.update(len(batch))
print("Tokenization complete")
import gensim
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)
print("model built")
model.build_vocab(data['TEXT'])
print("vocab built")
model.train(data['TEXT'], total_examples=model.corpus_count, epochs=model.epochs)
print("model trained")
model.save("word2vec.model")
print("model saved")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arsh0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Tokenizing data: 100%|██████████| 2249698/2249698 [10:24<00:00, 3407.45it/s]

Tokenization complete


Tokenizing data: 100%|██████████| 2249698/2249698 [10:37<00:00, 3407.45it/s]

model built
vocab built
model trained
model saved


In [2]:
import gensim
review_text = data.TEXT.apply(gensim.utils.simple_preprocess)
print("review text processed")
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)
print("model built")
import numpy as np
from tqdm import tqdm

w2v_model = gensim.models.Word2Vec.load('word2vec.model')
print("model loaded")
# Define a function to transform a list of words into a vector
def text_to_vector(text):
    vector = np.zeros(w2v_model.vector_size)
    num_words = 0
    for word in text:
        if word in w2v_model.wv:
            if word == 'tulip':
                print("tulip found")
            vector += w2v_model.wv[word]
            num_words += 1
    if num_words > 0:
        vector /= num_words
    return vector

# Apply the function to your text data to transform it into vectors
data['vectors'] = review_text.apply(lambda x: text_to_vector(x))

review text processed
model built
model loaded
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip found
tulip

In [5]:
data.to_csv('word2vec.csv', index=False)

In [5]:
print("start")
model.build_vocab(review_text, progress_per = 1000)
print("Vocab built")
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)
print("Model trained")
model.save("word2vec.model")
print("Model saved")

start
Vocab built
Model trained
Model saved


In [1]:
model.wv.similar_by_word('shoes')

NameError: name 'model' is not defined

In [3]:
import gensim
import numpy as np
from tqdm import tqdm

w2v_model = gensim.models.Word2Vec.load('word2vec.model')
# Define a function to transform a list of words into a vector
def text_to_vector(text):
    vector = np.zeros(w2v_model.vector_size)
    num_words = 0
    for word in text:
        print(word)
        print(word in w2v_model.wv)
        if word in w2v_model.wv:
            print("reached")
            vector += w2v_model.wv[word]
            num_words += 1
    if num_words > 0:
        vector /= num_words
    return vector

# Apply the function to your text data to transform it into vectors
data['vectors'] = review_text.apply(lambda x: text_to_vector(x))


In [7]:
data['vectors']

0          [ 2.76438198  1.03101277  1.43711287  0.188024...
1          [ 1.63797701  0.9281052   0.29943667  0.079936...
2          [ 0.49348591  1.00943766  2.06150385 -1.536642...
3          [ 2.53543026  1.13600364  0.16388655 -0.308092...
4          [ 0.81451654  0.24160927 -1.17535807  1.311491...
                                 ...                        
2249693    [ 1.08165692  0.5752942   0.76280622  0.263596...
2249694    [-0.60974843  1.56597216  0.98190239  1.656604...
2249695    [ 0.53458225  0.52373439  0.6412862   0.023277...
2249696    [-1.10309576 -0.65143969  1.57894686  1.319372...
2249697    [-6.39604987e-01 -2.28323179e-01  2.17671043e+...
Name: vectors, Length: 2249698, dtype: object

In [2]:
from sklearn.preprocessing import normalize
import numpy as np
def convert_string_to_array(vector_string):
    # remove square brackets
    vector_string = vector_string[2:-2]
    # split string into array of floats
    vector_array = np.array([float(num) for num in vector_string.split()])
    # reshape array to 2D array with single row
    vector_array = vector_array.reshape(1, -1)
    # normalize vectors in the array
    vector_array = normalize(vector_array)
    return vector_array
data['vectors'] = data['vectors'].apply(convert_string_to_array)

In [33]:
data.to_csv('word2vec.csv', index=False)

In [82]:
data.head()

,PRODUCT_LENGTH,vectors,vectors_1,vectors_2,vectors_3,vectors_4,vectors_5,vectors_6,vectors_7,vectors_8,...,vectors_91,vectors_92,vectors_93,vectors_94,vectors_95,vectors_96,vectors_97,vectors_98,vectors_99,vectors_100
0,2125.980000,"[[0.24486599278792262, 0.09132600607643815, 0....",0.113415,0.036357,0.034459,-0.015105,0.021919,-0.048887,-0.054492,-0.032069,...,-0.118073,-0.035667,0.01725,-0.06078,0.213969,0.110116,-0.009882,-0.035154,0.009612,-0.05306
1,393.700000,"[[0.12145305302285102, 0.06881733344131842, 0....",0.113415,0.036357,0.034459,-0.015105,0.021919,-0.048887,-0.054492,-0.032069,...,-0.118073,-0.035667,0.01725,-0.06078,0.213969,0.110116,-0.009882,-0.035154,0.009612,-0.05306
2,748.031495,"[[0.03937756535735372, 0.08054778591515246, 0....",0.113415,0.036357,0.034459,-0.015105,0.021919,-0.048887,-0.054492,-0.032069,...,-0.118073,-0.035667,0.01725,-0.06078,0.213969,0.110116,-0.009882,-0.035154,0.009612,-0.05306
3,787.401574,"[[0.21314571451127345, 0.09550028307038012, 0....",0.113415,0.036357,0.034459,-0.015105,0.021919,-0.048887,-0.054492,-0.032069,...,-0.118073,-0.035667,0.01725,-0.06078,0.213969,0.110116,-0.009882,-0.035154,0.009612,-0.05306
4,598.424000,"[[0.07313064166405209, 0.02169267298989804, -0...",0.113415,0.036357,0.034459,-0.015105,0.021919,-0.048887,-0.054492,-0.032069,...,-0.118073,-0.035667,0.01725,-0.06078,0.213969,0.110116,-0.009882,-0.035154,0.009612,-0.05306


In [3]:
nparr = np.array(data.vectors.to_list())
nparr = nparr.squeeze(axis=1)

In [4]:
nparr

array([[ 0.24486599,  0.09132601,  0.12729792, ...,  0.04174056,
         0.01596737,  0.06698156],
       [ 0.12145305,  0.06881733,  0.02220269, ..., -0.02544028,
         0.05622753,  0.09478858],
       [ 0.03937757,  0.08054779,  0.1644971 , ...,  0.00674603,
        -0.1030179 , -0.10387118],
       ...,
       [ 0.05688426,  0.05572996,  0.0682385 , ..., -0.06855369,
        -0.1818304 , -0.08117015],
       [-0.08860804, -0.052328  ,  0.12683158, ...,  0.03183769,
        -0.20036051, -0.10950936],
       [-0.06339921, -0.02263195,  0.21576087, ..., -0.09146177,
         0.0636594 , -0.16181896]])

In [12]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
with tf.device('/gpu:0'):
    # split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(nparr, data['PRODUCT_LENGTH'], test_size=0.2)

    # create a neural network model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])

    # compile the model
    model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

    # train the model
    model.fit(X_train, y_train, epochs=5, batch_size=64)

    # predict on test data
    y_pred = model.predict(X_test)

    # calculate evaluation metric
    score = max(0, 100 * (1 - metrics.mean_absolute_percentage_error(y_test, y_pred)))

    print('Score:', score)


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.